In [16]:
#Adjustments AI 
#Sample data creator


In [28]:
#Shared common config
SpecialFields=["AdjustmentType","AdjustmentName","AdjustedField"]
AuditFields=["OldVal","NewVal"]
SampleFields=["CustomerID","CustomerType","RiskCategory","IsDefaulted","CountryCode"]
NumericFields=["Drawn","Undrawn"]
RatioFields=["PD"]
AdjustmentTypes=["Amend"]
AdjustmentNames=["GRC_EndMonth","Fix_Broken_Batch","Credit_Uplift","Defaulting_Reclassify","Default"]
AdjustedFields=SampleFields
CustomerID=["CUST238947","CUST563201","CUST981372","CUST450896","CUST712035","CUST103289","CUST665428","CUST359147","CUST804231","CUST192654"]
CustomerType=["Retail","Corporate","SME"]
RiskCategory=["Low","Medium","High"]
IsDefaulted=["0","1"]
CountryCode=["US","GB","DE"]
FieldMapping={"CustomerID":CustomerID,"CustomerType":CustomerType,"RiskCategory":RiskCategory,"IsDefaulted":IsDefaulted,"CountryCode":CountryCode}

#Define adjustment rules
AdjustmentRules=[]
AdjustmentRules.append([[{"CustomerType":"Corporate"},{"RiskCategory":"High"}],{"PD",0.75},"Market Downturn"])


In [34]:
import pandas as pd
import numpy as np

#Sample Data Generator
records=1000

#Hide warmings from pandas
pd.options.mode.chained_assignment = None  # default='warn'

def GetRandomFieldValue(field):
    return np.random.choice(FieldMapping[field],1)[0]

#Checks if this row matches any adjustment rule, if so returns the [{fieldToUpdate:NewValue},AdjustmentComment]
def ValidateGetAdjustmentRule(row):
    for rule in AdjustmentRules:
        criteria=rule[0]
        adjustment=rule[1]

        #Check all criteria match
        allMatch=True
        for crit in criteria:
            #Split crit dictionary into key and value
            critField=list(crit.keys())[0]
            critValue=crit[critField]

            #Check if row matched
            if row[critField]!=critValue:
                allMatch=False

    return allMatch,adjustment


df=pd.DataFrame(columns=SampleFields+SpecialFields+AuditFields)
for mapping in FieldMapping:
    df[mapping]=np.random.choice(FieldMapping[mapping],records)
df["AdjustmentType"]=np.random.choice(AdjustmentTypes,records)
df["AdjustmentName"]=np.random.choice(AdjustmentNames,records)
df["AdjustedField"]=np.random.choice(AdjustedFields,records)
for field in NumericFields:
    #Insert new column after first 1
    df[field]=np.random.randint(1000000, size=records)
for field in RatioFields:
    df[field]=np.random.randint(100, size=records)/100

#Add in audit fields
df["OldVal"]=np.nan
df["NewVal"]=np.nan

#Loop over each row and generate a new value for Oldval based on field
for index, row in df.iterrows():
    adjField=row["AdjustedField"]

    #Check if this row matches any adjustment rule
    matchesRule,rule = ValidateGetAdjustmentRule(row)

    #APPLY ADJUTSMENTS--------------------------------------
    if adjField not in NumericFields and adjField not in RatioFields:

        if matchesRule:
            df["AdjustmentType"][index]="DERP"
            df["OldVal"][index]=row[]
            df["NewVal"][index]=GetRandomFieldValue(adjField)
        else:
            #Random "noise" adjustments
            df["OldVal"][index]=row[adjField]
            df["NewVal"][index]=GetRandomFieldValue(adjField)

# #Reshuffle the numeric fields to the start
df=df.reindex(columns=SpecialFields+NumericFields+RatioFields+SampleFields+AuditFields)

#Remove OldVal Field
df=df.drop(columns=["OldVal"])

# #Write to CSV file
df.to_csv("SampleData.csv",index=False)
df


,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,GRC_EndMonth,CountryCode,352883,417011,0.29,CUST563201,SME,Medium,0,US,GB
1,Amend,GRC_EndMonth,RiskCategory,372584,396770,0.55,CUST192654,Corporate,Medium,1,GB,Low
2,DERP,GRC_EndMonth,RiskCategory,85891,563156,0.59,CUST563201,Corporate,High,0,GB,NaN
3,Amend,Default,CustomerID,301090,589197,0.49,CUST712035,Retail,High,0,GB,CUST804231
4,Amend,Credit_Uplift,IsDefaulted,454232,898762,0.91,CUST665428,Corporate,Low,1,GB,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Amend,Default,CustomerID,393434,133986,0.70,CUST450896,Corporate,Medium,0,GB,CUST103289
996,Amend,Default,RiskCategory,194891,293619,0.25,CUST103289,SME,Medium,1,US,High
997,Amend,Default,CustomerID,68742,648091,0.62,CUST450896,SME,High,1,US,CUST804231
998,Amend,Credit_Uplift,CustomerID,309808,988588,0.16,CUST192654,SME,Low,1,US,CUST712035


3